In [10]:
from pydoc import doc
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.vectorstores.faiss import FAISS
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore


cached_dir = LocalFileStore("./.cache/")

llm = ChatOpenAI(
    temperature=0.1
)
loader = UnstructuredFileLoader("./files/documents.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size = 1000,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cached_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Please answer the question only with given context below. If you don't know about the question, don't make a answer.{context}"),
        ("human", "{question}"),
    ]
)

chain =  {"context":retriever, "question":RunnablePassthrough()} |prompt | llm

chain.invoke("Is Aaronson Guilty?")

AIMessage(content='Yes, according to the text, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.')

In [11]:
chain.invoke("What kind of message did Aaronson write?")


AIMessage(content='FREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE')

In [12]:
chain.invoke("Who is Julia")


AIMessage(content='Julia is a character in the text who is associated with Winston.')